## Lyrics Generation

### PRELIM - Creating the dataset

For the first step, we use the cleansed_id.txt file that we download from the lakh pianoroll dataset <a href = "https://salu133445.github.io/lakh-pianoroll-dataset/dataset">here.</a> This file is converted into a datafrae of two columns. The first column stores the values of the LPD (lakh pianoroll id) (*I am still not sure about what exactly this is*) and the second column stores the track_ids of the songs which is relevant for the million song dataset (MSD).

In [2]:
#importing the required datasets
import os
import pandas as pd
import numpy as np
import sqlite3

In [3]:
data_path = '/Users/harshitachadha/Desktop/data'
cleansed_ids = pd.read_csv(os.path.join(data_path,'cleansed_ids.txt'), delimiter = '    ', header = None)
cleansed_ids.columns =['LPD_ID', 'MSD_ID']
cleansed_ids.head()

<ipython-input-3-7c697c0f4ad2>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  cleansed_ids = pd.read_csv(os.path.join(data_path,'cleansed_ids.txt'), delimiter = '    ', header = None)


,LPD_ID,MSD_ID
0,cd3b9c8bb118575bcd712cffdba85fce,TRRNARX128F4264AEB
1,d8392424ea57a0fe6f65447680924d37,TRWMHMP128EF34293F
2,468be2f5dd31a1ba444b8018d8e8c7ad,TRWOLRE128F427D710
3,e6189e77aaca069eeaee516d1dd49995,TRAPFCP128F92D772E
4,cab3e200b884e453f9c035f4f28a2da7,TRBOEFO128F92FC62E


After this we refer to the MSD <a heref = "http://millionsongdataset.com/pages/getting-dataset/"> here</a> and download additional file number 8 which is the most metadata for all the tracks in the million song database. The data is in the form of an sqllite database with one table called 'Songs'. We convert this data into a dataframe.

In [6]:
cnx = sqlite3.connect(os.path.join(data_path,'track_metadata.db'))
df = pd.read_sql_query("SELECT * FROM songs", cnx)
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 14 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   track_id            1000000 non-null  object 
 1   title               1000000 non-null  object 
 2   song_id             1000000 non-null  object 
 3   release             1000000 non-null  object 
 4   artist_id           1000000 non-null  object 
 5   artist_mbid         1000000 non-null  object 
 6   artist_name         1000000 non-null  object 
 7   duration            1000000 non-null  float64
 8   artist_familiarity  1000000 non-null  float64
 9   artist_hotttnesss   1000000 non-null  float64
 10  year                1000000 non-null  int64  
 11  track_7digitalid    1000000 non-null  int64  
 12  shs_perf            1000000 non-null  int64  
 13  shs_work            1000000 non-null  int64  
dtypes: float64(3), int64(4), object(7)
memory usage: 106.8+ MB


In [7]:
#looking at the dataset
df.head()

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,TRMMMYQ128F932D901,Silent Night,SOQMMHC12AB0180CB8,Monster Ballads X-Mas,ARYZTJS1187B98C555,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,252.05506,0.649822,0.394032,2003,7032331,-1,0
1,TRMMMKD128F425225D,Tanssi vaan,SOVFVAK12A8C1350D9,Karkuteillä,ARMVN3U1187FB3A1EB,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,156.55138,0.439604,0.356992,1995,1514808,-1,0
2,TRMMMRX128F93187D9,No One Could Ever,SOGTUKN12AB017F4F1,Butter,ARGEKB01187FB50750,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,138.97098,0.643681,0.437504,2006,6945353,-1,0
3,TRMMMCH128F425532C,Si Vos Querés,SOBNYVR12A8C13558C,De Culo,ARNWYLR1187B9B2F9C,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,145.05751,0.448501,0.372349,2003,2168257,-1,0
4,TRMMMWA128F426B589,Tangle Of Aspens,SOHSBXH12A8C13B0DF,Rene Ablaze Presents Winter Sessions,AREQDTE1269FB37231,,Der Mystic,514.29832,0.000000,0.000000,0,2264873,-1,0


The next step is to match the track_ids that exist in the LPD dataset to the track_ids in the MSD dataframe above to obtain a relevant subset of data. This subset should have 21,425 values.

In [8]:
lpd_subset = df[df['track_id'].isin(cleansed_ids['MSD_ID'])]
len(lpd_subset)

21425

We now have a datframe that contains the metadata for all the pianoroll tracks in the LPD-5-cleansed dataset that we use in stage 3 of the pipeline. We now parse the lpd_subset dataset and contact the musixmatch api to get lyrics data. For all the songs that have lyrics we create a new dataset called pip2_subset. The function below achieves this.

In [37]:
#first we reset the index of the lpd_subset data and observe the results
lpd_subset.reset_index(inplace = True)
lpd_subset.head()

,index,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,32,TRMMMQN128F4238509,Raspberry Beret (LP Version),SOGNNYL12A6D4F910B,The Hits/The B-Sides,ARJ3CTF1187B9A1F2E,4c8ead39-b9df-4c56-a27c-51bc049cfd48,Prince & The Revolution,212.06159,0.684808,0.485445,0,1381741,29768,29768
1,98,TRMMWTG128F4283F07,You Needed Me,SOZTLAR12A8C1368A9,Wind Beneath My Wings,ARVP8NN1187FB3C1D3,80d07b5b-35b1-4d58-a72e-49ac98cba13f,Roger Whittaker,194.53342,0.586746,0.433199,1994,3296492,-1,0
2,122,TRMMGFX128F92E0CA1,Words - Re-Recording,SOQAWRX12A8C140FB3,Jeannie C. Riley Selected Hits,ARJOEGM1187B9B8F24,79251d77-dd72-46d3-a324-61c27760ce25,Jeannie C. Riley,181.99465,0.505952,0.373235,0,5271769,-1,0
3,168,TRMMHXK128F42AF4AC,Skase,SOWKZGU12AF72A2B4A,Christos Dantis Best,AROGWRA122988FEE45,,Christos Dantis,220.96934,0.480752,0.417593,0,3476504,-1,0
4,397,TRMMBOC12903CEB46E,Drumstern (BONUS TRACK),SOLPHZY12AC468ABA8,Sonar Kollektiv - Elektrish Compilation 2,ARK6OTG1187FB3EE00,5a2cda42-e4c1-4bff-824b-bd1b0a2fb6cf,Marcus Worgull,580.41424,0.506928,0.311226,0,8110539,-1,0


In [38]:
from musixmatch import Musixmatch
musixmatch = Musixmatch('8cbc3c6577efc1275fbdfd935e8d84b9')

In [74]:
#pip2_subset = pd.DataFrame(columns = ['track_id', 'title', 'artist_name','Lyrics'])
counter = 19362
for index, row in lpd_subset[19362:].iterrows():
    counter = counter+1
    print(counter)
    lyrics = musixmatch.matcher_lyrics_get(row['title'], row['artist_name'])
    status_var = lyrics["message"]['header']['status_code']
    if status_var == 200:
        temp = {'track_id': row['track_id'], 'title': row['title'], 
                'artist_name':row['artist_name'],'Lyrics':lyrics["message"]["body"]["lyrics"]["lyrics_body"]}
        pip2_subset = pip2_subset.append(temp, ignore_index = True)
    else:
        continue

19363
19364
19365
19366
19367
19368
19369
19370
19371
19372
19373
19374
19375
19376
19377
19378
19379
19380
19381
19382
19383
19384
19385
19386
19387
19388
19389
19390
19391
19392
19393
19394
19395
19396
19397
19398
19399
19400
19401
19402
19403
19404
19405
19406
19407
19408
19409
19410
19411
19412
19413
19414
19415
19416
19417
19418
19419
19420
19421
19422
19423
19424
19425
19426
19427
19428
19429
19430
19431
19432
19433
19434
19435
19436
19437
19438
19439
19440
19441
19442
19443
19444
19445
19446
19447
19448
19449
19450
19451
19452
19453
19454
19455
19456
19457
19458
19459
19460
19461
19462
19463
19464
19465
19466
19467
19468
19469
19470
19471
19472
19473
19474
19475
19476
19477
19478
19479
19480
19481
19482
19483
19484
19485
19486
19487
19488
19489
19490
19491
19492
19493
19494
19495
19496
19497
19498
19499
19500
19501
19502
19503
19504
19505
19506
19507
19508
19509
19510
19511
19512
19513
19514
19515
19516
19517
19518
19519
19520
19521
19522
19523
19524
19525
19526
19527
19528
1952

20729
20730
20731
20732
20733
20734
20735
20736
20737
20738
20739
20740
20741
20742
20743
20744
20745
20746
20747
20748
20749
20750
20751
20752
20753
20754
20755
20756
20757
20758
20759
20760
20761
20762
20763
20764
20765
20766
20767
20768
20769
20770
20771
20772
20773
20774
20775
20776
20777
20778
20779
20780
20781
20782
20783
20784
20785
20786
20787
20788
20789
20790
20791
20792
20793
20794
20795
20796
20797
20798
20799
20800
20801
20802
20803
20804
20805
20806
20807
20808
20809
20810
20811
20812
20813
20814
20815
20816
20817
20818
20819
20820
20821
20822
20823
20824
20825
20826
20827
20828
20829
20830
20831
20832
20833
20834
20835
20836
20837
20838
20839
20840
20841
20842
20843
20844
20845
20846
20847
20848
20849
20850
20851
20852
20853
20854
20855
20856
20857
20858
20859
20860
20861
20862
20863
20864
20865
20866
20867
20868
20869
20870
20871
20872
20873
20874
20875
20876
20877
20878
20879
20880
20881
20882
20883
20884
20885
20886
20887
20888
20889
20890
20891
20892
20893
20894
2089

In [78]:
pip2_subset.to_csv('/Users/harshitachadha/Desktop/data/matched_lyrics.csv', index=False)

In the above section of code, we create the lpd-5 dataset's subset for those songs where the lyrics are available. We store this data as a csv file called matched_lyrics.csv in the data directory. This will be used in stage 2 of the pipeline to identify the priming sequence.

### PART1 - Importing Dataset

In [84]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import string, os 
import tensorflow as tf

# keras module for building LSTM 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dropout, LSTM, Dense, Bidirectional 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt
import seaborn as sns

The data for lyrics generation comes from kaggle linked <a href ='https://www.kaggle.com/datasets/neisse/scrapped-lyrics-from-6-genres'> here.</a>

In [105]:
#imprting the data downloaded from the link above
df_lstm = pd.read_csv("/Users/harshitachadha/Desktop/data/lyrics-data.csv")

In [106]:
df_lstm.head()

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt


This dataset has songs from many language, but here we will focus only on english language lyrics first. So we refine this dataset so as to only get the english language songs. Initially there are about 3 lakh 80 thousand songs in this dataset.

In [107]:
#first we drop columns Alink, SLink, and SName so that the dataframe only has lyrics column
df_lstm.drop(['ALink','SName','SLink'],axis=1,inplace=True)
df_lstm.head()

,Lyric,language
0,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,A minha sorte grande foi você cair do céu\nMin...,pt


In [109]:
initial_len = len(df_lstm) #obtaining the initial number of lyrics from all languages
#dropping all non english songs
df_lstm_en = df_lstm[df_lstm['language']=='en']
print('The percentage of lyrics retained is {}'. format((len(df_lstm_en)/initial_len)*100))

The percentage of lyrics retained is 50.48653571306369


In [111]:
#visualising the dataset obtained 
df_lstm_en.head()

,Lyric,language
69,I feel so unsure\nAs I take your hand and lead...,en
86,"Don't let them fool, ya\nOr even try to school...",en
88,"Baby, let's cruise, away from here\nDon't be c...",en
111,"Know it sounds funny\nBut, I just can't stand ...",en
140,You've got that look again\nThe one I hoped I ...,en


In [113]:
#we now reset the index and drop the language column also
df_lstm_en.reset_index(inplace=True)
df_lstm_en.drop(['language'],axis=1,inplace=True)
df_lstm_en.drop(['index'],axis=1,inplace=True)
df_lstm_en.head()

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Lyric
0,I feel so unsure\nAs I take your hand and lead...
1,"Don't let them fool, ya\nOr even try to school..."
2,"Baby, let's cruise, away from here\nDon't be c..."
3,"Know it sounds funny\nBut, I just can't stand ..."
4,You've got that look again\nThe one I hoped I ...


In [116]:
#adding a column that contains the number of words in the lyric sequence
df_lstm_en['Number_of_words'] = df_lstm_en['Lyric'].apply(lambda x:len(str(x).split()))
df_lstm_en.head()

<ipython-input-116-543b86328647>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lstm_en['Number_of_words'] = df_lstm_en['Lyric'].apply(lambda x:len(str(x).split()))


,Lyric,Number_of_words
0,I feel so unsure\nAs I take your hand and lead...,299
1,"Don't let them fool, ya\nOr even try to school...",161
2,"Baby, let's cruise, away from here\nDon't be c...",159
3,"Know it sounds funny\nBut, I just can't stand ...",175
4,You've got that look again\nThe one I hoped I ...,215


In [117]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_lstm_en['Lyric'].astype(str).str.lower())
total_words = len(tokenizer.word_index)+1
tokenized_sentences = tokenizer.texts_to_sequences(df_lstm_en['Lyric'].astype(str))
tokenized_sentences[0]

[2,
 75,
 23,
 6323,
 73,
 2,
 69,
 12,
 232,
 5,
 647,
 3,
 4,
 1,
 244,
 465,
 73,
 1,
 340,
 1710,
 173,
 10,
 12,
 126,
 1344,
 4,
 134,
 6,
 1078,
 2041,
 5,
 16,
 294,
 576,
 4018,
 15,
 52,
 78,
 244,
 115,
 1752,
 453,
 62,
 36,
 29,
 1094,
 368,
 32,
 409,
 4,
 880,
 2,
 24,
 48,
 55,
 6,
 481,
 1531,
 695,
 147,
 164,
 4,
 2644,
 6,
 347,
 5,
 789,
 1,
 425,
 14,
 94,
 86,
 1065,
 23,
 15,
 52,
 78,
 244,
 115,
 1,
 66,
 2,
 2535,
 27,
 3,
 46,
 38,
 52,
 2730,
 1,
 3924,
 2709,
 11,
 6,
 109,
 347,
 4,
 1,
 88,
 5,
 134,
 3375,
 17,
 422,
 116,
 29,
 1701,
 10,
 1,
 383,
 252,
 17,
 16,
 187,
 138,
 15,
 52,
 78,
 244,
 115,
 1752,
 453,
 62,
 36,
 29,
 1094,
 368,
 32,
 409,
 4,
 880,
 2,
 24,
 48,
 55,
 6,
 481,
 1531,
 695,
 147,
 164,
 4,
 2644,
 6,
 347,
 5,
 789,
 30,
 425,
 14,
 94,
 86,
 1065,
 23,
 15,
 52,
 78,
 244,
 115,
 1,
 66,
 2,
 2535,
 27,
 3,
 52,
 210,
 12,
 22,
 161,
 1,
 340,
 457,
 23,
 696,
 2,
 293,
 14,
 18,
 92,
 330,
 30,
 930,
 280,
 32,
 147,
 3

In [ ]:
# Slash sequences into n gram sequence
input_sequences = list()
for i in tokenized_sentences:
    for t in range(1, len(i)):
        n_gram_sequence = i[:t+1]
        input_sequences.append(n_gram_sequence)
        
# Pre padding
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
# create predictors and label
X, labels = input_sequences[:,:-1],input_sequences[:,-1]
y = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
# create model
model = Sequential()
model.add(Embedding(total_words, 40, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(250)))
model.add(Dropout(0.1))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
earlystop = EarlyStopping(monitor='loss', min_delta=0, patience=3, verbose=0, mode='auto')
history = model.fit(X, y, epochs=10, verbose=1, callbacks=[earlystop])

In [ ]:
# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
def complete_this_song(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [ ]:
complete_this_song("the sky is blue", 40)

In [ ]:
# save model
from tensorflow.keras.models import load_model
model.save('lstm_lyric_comp1.h5')